In [29]:
import json
import pandas as pd
from a2ginputstream import A2GClient
import random
import time
from datetime import datetime
from datetime import timedelta
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

## Read Data

In [30]:
USER_PAT, IKEY = "8f8ff571c7344ce9b6fae09f1ff9c5b1", "1c01a94bea5948faabda"
client = A2GClient(token=USER_PAT)
time_now = datetime.utcnow().replace(
    microsecond=0,
    second=0,
    minute=0
)

start_hour = time_now
end_hour = time_now + timedelta(hours=1)

movements_data:list[dict] = client.find(IKEY, {"start_on": {"$gte":start_hour}, "end_on":{"$lte":end_hour}}, delete_id=True)

2024-10-28 17:34:24,364 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-10-28 17:34:24,803 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/1c01a94bea5948faabda HTTP/1.1" 200 None
2024-10-28 17:34:24,814 - INFO - Getting inputstream 1c01a94bea5948faabda from A2G...
2024-10-28 17:34:24,817 - INFO - Downloading data from A2G...
2024-10-28 17:34:24,820 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-28 17:34:25,295 - DEBUG - https://v2streams.a2g.io:443 "POST /QueryData/ExecutionPlanningFind HTTP/1.1" 200 119
2024-10-28 17:34:25,296 - INFO - The query stages are COLLSCAN
2024-10-28 17:34:25,297 - INFO - The index used in query is None
2024-10-28 17:34:25,297 - INFO - Total documents to download 31.
2024-10-28 17:34:25,298 - INFO - Batch 0/1
2024-10-28 17:34:25,302 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-28 17:34:25,853 - DEBUG - https://v2streams.a2g.io:443 "POST /QueryData/FindAll?page=1

## Consolidate data by user and machine

In [31]:
df_movements = pd.DataFrame(movements_data)
df_users = df_movements.groupby(by="name").sum()
df_machines = df_movements.groupby(by="machines").sum()

df_machines.reset_index(inplace=True)
df_users.reset_index(inplace=True)

df_users["time"] = time_now.isoformat()
df_machines["time"] = time_now.isoformat()

C:\Users\DaniloAraneda\AppData\Local\Temp\ipykernel_49452\3027444034.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_users = df_movements.groupby(by="name").sum()
C:\Users\DaniloAraneda\AppData\Local\Temp\ipykernel_49452\3027444034.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_machines = df_movements.groupby(by="machines").sum()


In [32]:
df_machines

,machines,ton,time
0,C301,628,2024-10-28T20:00:00
1,C302,657,2024-10-28T20:00:00
2,C303,629,2024-10-28T20:00:00


In [33]:
df_users

,name,ton,time
0,Ahbraham Smith,377,2024-10-28T20:00:00
1,Jane Doe,274,2024-10-28T20:00:00
2,John Doe,226,2024-10-28T20:00:00
3,John Smith,311,2024-10-28T20:00:00
4,Richard Alfredo,347,2024-10-28T20:00:00
5,Smith Alfred,379,2024-10-28T20:00:00


## Save in inputstream

In [34]:
IKEY_machine = "574dd3c216924478a03e"
client.insert_data(IKEY_machine, df_machines.to_dict(orient="records"))

IKEY_users = "0fa6688ad39f4622bdd9"
client.insert_data(IKEY_users, df_users.to_dict(orient="records"))

2024-10-28 17:34:36,676 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-10-28 17:34:37,108 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/574dd3c216924478a03e HTTP/1.1" 200 None
2024-10-28 17:34:37,119 - INFO - Getting inputstream 574dd3c216924478a03e from A2G...
2024-10-28 17:34:37,121 - WARNING - Inputstream discovered
2024-10-28 17:34:37,124 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-28 17:34:37,580 - DEBUG - https://v2streams.a2g.io:443 "POST /Data/Insert HTTP/1.1" 200 None
2024-10-28 17:34:37,588 - INFO - batch 1, docs: [0 - 3] - 200 - {"message":"Data Received Success, DataToyMachine inputstream","data":null,"success":true,"errors":null}
2024-10-28 17:34:37,591 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-10-28 17:34:38,033 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/0fa6688ad39f4622bdd9 HTTP/1.1" 200 None
2024-10-28 17:34:38,048 - INFO - Getting inputstrea